In [1]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print(device)

cuda:0


### Dataset

In [2]:
train_batch_size = 64
test_batch_size = 64

train_set = datasets.MNIST('./data',train=True, download=True, transform=transforms.ToTensor())
trainloader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True)

test_set = datasets.MNIST('./data',train=False, download=True, transform=transforms.ToTensor())
testloader = DataLoader(test_set, batch_size= test_batch_size, shuffle= True)

### Define train&test accuracy

In [3]:
def train_accuracy(model):
    model.eval()
    train_loss = 0
    correct = 0
    for data, target in trainloader:
        
        data, target = data.cuda(), target.cuda()
        output = model(data)
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    print('\ntrain set: Accuracy: {}/{} ({:.2f}%)\n'.format(correct,len(trainloader.dataset),100. * correct / len(trainloader.dataset)))

In [4]:
def test_accuracy(model):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in testloader:
        
        data, target = data.cuda(), target.cuda()
        output = model(data)
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    print('\nTest set: Accuracy: {}/{} ({:.2f}%)\n'.format(correct,len(testloader.dataset),100. * correct / len(testloader.dataset)))

### Define TeacherNet

In [5]:
class Teacher_Net(nn.Module):
    
    def __init__(self):
        super(Teacher_Net, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out
    
    
model_T = Teacher_Net().to(device)

### Training TeacherNet

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_T.parameters(),lr=0.001)

def train_teacherNet(model,epoch):
    
    for i in range(epoch):
        
        for batch_idx, (data, target) in enumerate(trainloader):

            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()

            output = model_T(data)

            loss = loss_fn(output, target)

            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    i, batch_idx * len(data), len(trainloader.dataset),
                    100. * batch_idx / len(trainloader), loss.item()))

        torch.save(model.state_dict(), './teacher_weights.pt')

### Define Student Model

In [7]:
class Student_Net(nn.Module):
    
    def __init__(self):
        super(Student_Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 800)
        self.fc2 = nn.Linear(800, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x
    
model_S = Student_Net().to(device)

### Training Student Model

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_S.parameters(),lr=0.001)

def train_StudentNet(model,epoch):
    
    for i in range(epoch):
        for batch_idx, (data, target) in enumerate(trainloader):

            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()

            output = model_S(data)

            loss = loss_fn(output, target)

            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    i, batch_idx * len(data), len(trainloader.dataset),
                    100. * batch_idx / len(trainloader), loss.item()))

### Knowledge Distillation on Mnist

In [9]:
model_T.load_state_dict(torch.load('./teacher_weights.pt')) # load Teacher model weigths
model_S = Student_Net().to(device)

### Define loss total for knowledge distillation

In [10]:
def loss_total(outputT, outputS, target, T, K):
    
    outputT_log = F.log_softmax(outputT/T, dim=1) 
    outputS_log = F.log_softmax(outputS/T, dim=1)

    KLDivLoss = nn.KLDivLoss(reduction='batchmean')
    loss_kd = KLDivLoss(outputS_log, outputT_log) 

    loss_ce = nn.CrossEntropyLoss()(outputS, target)

    loss_total = loss_ce* (1. - K) + loss_kd * (T * T + K) 

    return loss_total

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_S.parameters(),lr=0.001)

def train_KD(model_S, model_T,epoch):
    
    for i in range(epoch):
        for batch_idx, (data, target) in enumerate(trainloader):

            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()

            outputS = model_S(data)

            outputT = model_T(data) # logit
            outputT = outputT.detach()

            loss = loss_total(outputT, outputS, target,5,0.3)

            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    i, batch_idx * len(data), len(trainloader.dataset),
                    100. * batch_idx / len(trainloader), loss.item()))